In [1]:
from param.arg_utils import get_args
from experiment import Experiment
import easydict
import gym
import pygame
import torch as th
from stable_baselines3 import A2C,PPO
from stable_baselines3.common.env_checker import check_env
from model.extractor import StackedImgStateExtractor, ImgStateExtractor

device = th.device("cuda:0" if th.cuda.is_available() else "cpu")

params = easydict.EasyDict({
        'env':'gym-2d-perception-v1',
        'gaze_method':'LookAhead',
        'render':False,
        'dt':0.1,
        'map_scale':10,
        'map_size':[640,480],
        'agent_number':10,
        'agent_max_speed':20,
        'agent_radius':10,
        'drone_max_speed':40,
        'drone_max_acceleration':15,
        'drone_radius':5,
        'drone_max_yaw_speed':80,
        'drone_view_depth' : 100,
        'drone_view_range': 90,
        'record': False,
        'pillar_number':3
    })

alg_params = {
    "policy_kwargs": dict(
        net_arch=[512, dict(pi=[256], vf=[256])],
        normalize_images=False,
        features_extractor_class=ImgStateExtractor,
        features_extractor_kwargs=dict(
            device=device, cnn_encoder_name="CnnEncoder"
        ),
    ),
}

env = gym.make('gym-2d-perception-v1', params=params) 
check_env(env)
model = PPO(
            "MultiInputPolicy",
            env,
            verbose=1,
            device=device,
            tensorboard_log='./experiment/log/',
            **alg_params
        )
model.learn(total_timesteps=30000)
pygame.display.quit()
vec_env = model.get_env()
vec_env.is_render = True
obs = vec_env.reset()

pygame 2.1.2 (SDL 2.0.16, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


/home/smoggy/anaconda3/envs/thesis/lib/python3.9/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))
/home/smoggy/anaconda3/envs/thesis/lib/python3.9/site-packages/stable_baselines3/common/env_checker.py:26: UserWarning: It seems that your observation  is an image but the `dtype` of your observation_space is not `np.uint8`. If your observation is not an image, we recommend you to flatten the observation to have only a 1D vector
  warnings.warn(


Using cuda:0 device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./experiment/log/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 498      |
|    ep_rew_mean     | 1.17e+03 |
| time/              |          |
|    fps             | 32       |
|    iterations      | 1        |
|    time_elapsed    | 63       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 390         |
|    ep_rew_mean          | 734         |
| time/                   |             |
|    fps                  | 36          |
|    iterations           | 2           |
|    time_elapsed         | 111         |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.016028605 |
|    clip_fraction        | 0.0958      |
|    clip_range          

In [3]:
params = easydict.EasyDict({
        'env':'gym-2d-perception-v1',
        'gaze_method':'LookAhead',
        'render':True,
        'dt':0.1,
        'map_scale':10,
        'map_size':[640,480],
        'agent_number':10,
        'agent_max_speed':20,
        'agent_radius':10,
        'drone_max_speed':40,
        'drone_max_acceleration':15,
        'drone_radius':5,
        'drone_max_yaw_speed':80,
        'drone_view_depth' : 100,
        'drone_view_range': 90,
        'record': False,
        'pillar_number':3
    })
env = gym.make('gym-2d-perception-v1', params=params) 
for i in range(1000):
    action, _state = model.predict(obs, deterministic=True)

    print(action)

    obs, reward, done, info = env.step(action)
    env.render()

/home/smoggy/anaconda3/envs/thesis/lib/python3.9/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


ValueError: Error: Unexpected observation shape (1, 33, 33) for Box environment, please use (1, 41, 41) or (n_env, 1, 41, 41) for the observation shape.